In [2]:
import pandas as pd
import numpy as np

In [3]:
all_results = pd.read_pickle("../results.pk")
keys = list(all_results.keys())
keys = [i for i in keys if "catastrophic" not in i]
parameters = [i.split(".")[1:] for i in keys]
parameters = [i for i in parameters if len(i) == 11]
parameters = np.array(parameters)
parameters = [list(np.unique(parameters[:,i])) for i in range(0,11)]
datasets = 0
modalities = 1
tasks = 2
training_strategy = 3
dropout = 5
label_scheme = 6
epochs = 8
kg = 10
for i in keys:
    new_key = '.'.join(i.split(".")[0:8]+i.split(".")[9:])
    all_results[new_key] = all_results[i]
    del all_results[i]
keys = list(all_results.keys())

In [4]:
import gradio as gr
import re


def predict(datasets,modalities,tasks,training_strategy,dropout,label_scheme,epochs,kg):
    key = f"results.{datasets}.{modalities}.{tasks}.{training_strategy}.its:1.{dropout}.{label_scheme}.{epochs}.256.{kg}"

    tasks = tasks.split(',')
    
    try:
      first_task = all_results[key][re.search("\'(.*)\'",tasks[0])[1]].apply(pd.Series).astype(float).iloc[:,:5]
      first_task.index = ['-'.join(i.split("-")[2:]) for i in first_task.index]
      first_task = first_task.sort_index().reset_index()
      second_task = pd.DataFrame()
      if len(tasks) == 2:
        second_task = all_results[key][re.search("\'(.*)\'",tasks[1])[1]].apply(pd.Series).astype(float).iloc[:,:5]
        second_task.index = ['-'.join(i.split("-")[2:]) for i in second_task.index]
        second_task = second_task.sort_index().reset_index()
      return first_task,second_task,"",""
    except Exception as msg:
        return pd.DataFrame(),pd.DataFrame(),"No results were found for that given combination of parameters","The avaliable combinations are: "+str(keys)

iface = gr.Interface(predict,
  [
    gr.inputs.Dropdown(choices=parameters[datasets]),
    gr.inputs.Dropdown(choices=parameters[modalities]),
    gr.inputs.Dropdown(choices=parameters[tasks]),
    gr.inputs.Dropdown(choices=parameters[training_strategy]),
    gr.inputs.Dropdown(choices=parameters[dropout]),
    gr.inputs.Dropdown(choices=parameters[label_scheme]),
    gr.inputs.Dropdown(choices=parameters[epochs]),
    gr.inputs.Dropdown(choices=parameters[kg],label="Graph topology")
  ],
  [
  "dataframe",
  "dataframe",
  "text",
  "text"
  ],  
  
  title="Select the parameters to see the results",
  allow_flagging=False,
  allow_screenshot=False,
  live=True

)

#iface.test_launch()

iface.launch()


Running on local URL:  http://127.0.0.1:7870/

To create a public link, set `share=True` in `launch()`.


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7870/', None)